# Previously

We finished saving and loading our models but we just download our dataset. In real life, most probably, that won't be the case. We want to process our own dataset.

# In this notebook, 

We'll dig in more to our Preliminary Step. We'll
- load our own dataset from a certain storage
- do Transforms to our Image Data
- get a detailed view of loading the data to be used in our model
- learn about Data Augmentation 

# Sample Dataset

We'll be using a [dataset of cat and dog photos](https://www.kaggle.com/c/dogs-vs-cats) available from Kaggle. Here are a couple example images:

![Dog and Cat](https://github.com/lbleal1/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/dog_cat.png?raw=true) 

**For raw datasets, you can expect:**
- you need to resize it for training (depending on the neural network)
- if it's few, you might wanna augment it (we'll discuss Data Augmentation later)

*This is different from the dataset we had before because those are already prepared for training. So beware of the needed preparation depending on the context of the data and your neural network.

***Remark:***
- These days it doesn't seem like a big accomplishment, but five years ago it was a serious challenge for computer vision systems.

In [13]:
# folks the helper package always goes in the way as with the previous notebooks
# so don't forget this one if you're running in colab

!wget https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/3bd7dea850e936d8cb44adda8200e4e2b5d627e3/intro-to-pytorch/helper.py

# if this doesn't still get through by running, reset your runtime and
# it will work

# also don't forget to change your Hardware Accelerator above to GPU for faster processing

--2020-01-11 15:34:08--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/3bd7dea850e936d8cb44adda8200e4e2b5d627e3/intro-to-pytorch/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2813 (2.7K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   2.75K  --.-KB/s    in 0s      

2020-01-11 15:34:09 (92.0 MB/s) - ‘helper.py’ saved [2813/2813]



In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms

import helper

# Basics: Loading Image Data, Transform, Loading Data for Training

##Conceptual

### Loading Image Data

`datasets.ImageFolder`
- used to load image data (see [documentation](http://pytorch.org/docs/master/torchvision/datasets.html#imagefolder))
-  expects the files and directories to be constructed like so:

```
root/dog/xxx.png
root/dog/xxy.png
root/dog/xxz.png

root/cat/123.png
root/cat/nsdf3.png
root/cat/asd932_.png
```

where each class has it's own directory (`cat` and `dog`) for the images.


 

### Transforms

`transform` 
- a sequence of processing steps built with the [`transforms`](http://pytorch.org/docs/master/torchvision/transforms.html) module from `torchvision`. ImageFolder
  *The images are then labeled with the class taken from the directory name. So here, the image `123.png` would be loaded with the class label `cat`. 
- done after loading in the data with `ImageFolder`
- this is used to let the images conform to the descriptions we want it to have such as:
  - we want it to have a uniform size, so we do
    - resize them with `transforms.Resize()`
    - or crop with `transforms.CenterCrop()`, `transforms.RandomResizedCrop()`, etc.
  - we want it to convert to tensors
    - so we'll use `transforms.ToTensor()`
  - we want to combine these transforms into pipeline with `transforms.Compose()`
    - which accepts a list of transforms and runs them in sequence.
- Example:
    ```python
transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

    ```

There are plenty of transforms available, I'll cover more in a bit and you can read through the [documentation](http://pytorch.org/docs/master/torchvision/transforms.html). 




### Loading your Data for Training with `DataLoader`

**[`DataLoader`](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader)**
- takes a dataset (such as you would get from `ImageFolder`)
- returns batches of images and the corresponding labels
- *You can set various parameters like the batch size and if the data is shuffled after each epoch.

```python
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
```

Here `dataloader` is a [generator](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/). To get data out of it, you need to loop through it or convert it to an iterator and call `next()`.

**recall:** We have an option to get per batch manually or go through all the batches
```python
# Get one batch
images, labels = next(iter(dataloader))

# Looping through it, get a batch on each loop 
for images, labels in dataloader:
    pass

```
 

## Coding the Basic Preliminaries

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
data_dir = '/content/drive/My Drive/Udacity-AI Notes/Notebooks/L5 Notebooks/assets/dogs-vs-cats/train'

transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])
dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [0]:
# Run this to test your data loader
images, labels = next(iter(dataloader))
helper.imshow(images[0], normalize=False)

If you loaded the data correctly, you should see something like this (your image will be different):

![Cat](https://github.com/lbleal1/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/cat_cropped.png?raw=true) 


# Data Augmentation


## Conceptual


**Data Augmentation**
- a common strategy for training neural networks is to introduce randomness in the input data itself
- for example, you can randomly rotate, mirror, scale, and/or crop your images during training. 
- this will help your network generalize as it's seeing the same images but in different locations, with different sizes, in different orientations, etc.

To randomly rotate, scale and crop, then flip your images you would define your transforms like this:

```python
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5, 0.5, 0.5], 
                                                            [0.5, 0.5, 0.5])])
```

`transforms.Normalize`
- you pass in a list of means and list of standard deviations, then the color channels are normalized like so
- Normalizing helps keep the network weights near zero which in turn makes backpropagation more stable. Without normalization, networks will tend to fail to learn.

```input[channel] = (input[channel] - mean[channel]) / std[channel]```

  - Subtracting `mean` centers the data around zero and dividing by `std` squishes the values to be between -1 and 1. 

***Remark: Some Guidelines for Transforms***

- You can find a list of all [the available transforms here](http://pytorch.org/docs/0.3.0/torchvision/transforms.html). 
- When you're testing however, you'll want to use images that aren't altered other than normalizing. 
- So, for validation/test images, you'll typically just resize and crop.


## Coding Data Augmentation

In [0]:
data_dir = '/content/drive/My Drive/Udacity-AI Notes/Notebooks/L5 Notebooks/assets/dogs-vs-cats'

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()]) 

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor()])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)

In [0]:
# change this to the trainloader or testloader 
data_iter = iter(testloader)

images, labels = next(data_iter)
fig, axes = plt.subplots(figsize=(10,4), ncols=4)
for ii in range(4):
    ax = axes[ii]
    helper.imshow(images[ii], ax=ax, normalize=False)

Your transformed images should look something like this.

<center>Training examples:</center>

![Train Examples](https://github.com/lbleal1/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/train_examples.png?raw=true) 

<center>Testing examples:</center>

![Test Examples](https://github.com/lbleal1/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/test_examples.png?raw=true) 

# At this point,

You should be able to load data for training and testing. Now, you should try building a network that can classify cats vs dogs. This is quite a bit more complicated than before with the MNIST and Fashion-MNIST datasets. To be honest, you probably won't get it to work with a fully-connected network, no matter how deep. These images have three color channels and at a higher resolution (so far you've seen 28x28 images which are tiny).



# Next Up!

We'll learn how to use a pre-trained network to build a model that can actually solve this problem. This will be the final notebook and we will use all what we've learned from these past 7 notebooks to solve it. Til the next notebook!